In [39]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,round,max

In [2]:
pyspark.__version__

'3.3.2'

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 18:51:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-06.parquet

--2023-03-05 18:50:50--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-06.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.225.189.175, 13.225.189.178, 13.225.189.130, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.225.189.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394114750 (376M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-06.parquet’

fhvhv_tripdata_2021 100%[===================>] 375.86M  88.5MB/s    in 4.3s    

2023-03-05 18:50:54 (87.3 MB/s) - ‘fhvhv_tripdata_2021-06.parquet’ saved [394114750/394114750]



In [7]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-06.parquet')

In [8]:
from pyspark.sql import types

In [9]:
df =df.select('hvfhs_license_num','dispatching_base_num','pickup_datetime','dropoff_datetime',\
          'PULocationID','DOLocationID','shared_request_flag').withColumnRenamed('shared_request_flag','SR_Flag')

In [10]:
df = df.repartition(12)

In [11]:
df.write.parquet('fhvhv/2021/06/')

In [44]:
df = spark.read.parquet('fhvhv/2021/06/')

In [45]:
df.filter("cast(pickup_datetime as date) = '2021-06-15'").count()

452470

In [46]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [47]:
df= df.withColumn('DiffInSeconds',col("dropoff_datetime").cast("long") - col('pickup_datetime').cast("long"))\
     .withColumn('DiffInMinutes',round(col('DiffInSeconds')/60))

In [48]:
df= df.withColumn('DiffInHours',round(col('DiffInMinutes')/60))

In [49]:
df.select(max(df.DiffInHours)).show()

+----------------+
|max(DiffInHours)|
+----------------+
|            67.0|
+----------------+



In [58]:
df_zone =  spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv').alias('df_zone')

In [57]:
df = df.alias("df")

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [62]:
df = df.join(df_zone,df_zone.LocationID ==df.PULocationID,'inner')

In [69]:
df.groupBy('Zone').count().sort(col("count").desc()).show(10)

+--------------------+------+
|                Zone| count|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
+--------------------+------+
only showing top 10 rows

